# Analyzing Electric Charging Stations in the United States

### Step 1: Load the data

Note: I sourced this data from the U.S. Department of Energy Alternative Fuels Data Center: https://afdc.energy.gov/stations#/find/nearest?country=US.  

Specifically, I downloaded a CSV of alternative fuel charging stations data, using the API here: https://developer.nrel.gov/docs/transportation/alt-fuel-stations-v1/all/#csv-output-format.


Importing the CSV properly took me a few attempts, as I had to do some cleaning and exclusion of an erroneous row. Here's the final code I used:

In [1]:
# Import packages
import pandas as pd
import matplotlib.pyplot as plt

# File path placeholders
file_path = 'path/to/your/dataset/filtered_alt_fuel_stations.csv'
cleaned_file_path = 'path/to/your/dataset/cleaned_filtered_alt_fuel_stations.csv'


# Read the file line by line to identify and remove problematic rows
clean_lines = []
with open(file_path, 'r') as file:
    for line in file:
        if '5295 Westview Drive SIDE' not in line: # Exclude rows where 'Fuel Type Code' is '5295 Westview Drive SIDE' (this is a known erroneous row)
            clean_lines.append(line)

# Write the cleaned lines to a temporary file
cleaned_file_path = 'path/to/your/dataset/cleaned_filtered_alt_fuel_stations.csv'
with open(cleaned_file_path, 'w') as file:
    file.writelines(clean_lines)

# Load the cleaned csv file into a DataFrame
df = pd.read_csv(cleaned_file_path, low_memory=False)

# Convert 'ZIP' to string and 'Longitude' to float
df['ZIP'] = df['ZIP'].astype(str)
df['Longitude'] = pd.to_numeric(df['Longitude'], errors='coerce')

# Parse the 'Open Date' column to datetime
df['Open Date'] = pd.to_datetime(df['Open Date'], errors='coerce')

# Use head to inspect
df.head()


,Fuel Type Code,Station Name,Street Address,City,State,ZIP,Latitude,Longitude,Owner Type Code,Open Date,Access Code,Facility Type,EV Workplace Charging
0,CNG,Spire - Montgomery Operations Center,2951 Chestnut St,Montgomery,AL,36107,32.367916,-86.267021,T,2010-12-01,private,STANDALONE_STATION,NaN
1,CNG,Metropolitan Atlanta Rapid Transit Authority,2424 Piedmont Rd NE,Atlanta,GA,30324,33.821911,-84.367461,LG,1996-12-15,private,FLEET_GARAGE,NaN
2,CNG,United Parcel Service,270 Marvin Miller Dr,Atlanta,GA,30336,33.760256,-84.543822,P,1997-01-01,private,STANDALONE_STATION,NaN
3,CNG,Arkansas Oklahoma Gas Corp,2100 S Waldron Rd,Fort Smith,AR,72903,35.362213,-94.375338,T,1997-01-01,public,UTILITY,NaN
4,CNG,Clean Energy - Logan International Airport,1000 Cottage St Ext,East Boston,MA,2128,42.374706,-71.026549,SG,1996-11-15,public,AIRPORT,NaN


### Step 2: Filter and prepare the data

Next, I'll filter for stations with electricity as their exclusive fuel type. (Note: some stations have multiple fuel types.)

I'll then add a separate 'Year' column, using the existing 'Open Date' column.

In [2]:
# Filter the DataFrame for rows where the Fuel Type Code is 'ELEC'
ev_data = df[df['Fuel Type Code'] == 'ELEC'].copy()

# Extract the year from the 'Open Date' column
ev_data['Open Date'] = pd.to_datetime(ev_data['Open Date'], errors='coerce')
ev_data['Year'] = ev_data['Open Date'].dt.year

# Drop rows with NaT years
ev_data = ev_data.dropna(subset=['Year'])

# Convert 'Year' to integer
ev_data['Year'] = ev_data['Year'].astype(int)

ev_data.head()


,Fuel Type Code,Station Name,Street Address,City,State,ZIP,Latitude,Longitude,Owner Type Code,Open Date,Access Code,Facility Type,EV Workplace Charging,Year
133,ELEC,LADWP - Truesdale Center,11797 Truesdale St,Sun Valley,CA,91352,34.248319,-118.387971,LG,1999-10-15,private,UTILITY,True,1999
134,ELEC,Los Angeles Convention Center,1201 S Figueroa St,Los Angeles,CA,90015,34.040539,-118.271387,P,1995-08-30,public,PARKING_GARAGE,False,1995
135,ELEC,LADWP - John Ferraro Building,111 N Hope St,Los Angeles,CA,90012,34.059133,-118.248589,LG,1999-10-15,private,UTILITY,True,1999
136,ELEC,LADWP - Haynes Power Plant,6801 E 2nd St,Long Beach,CA,90803,33.759802,-118.096665,LG,2018-05-01,private,UTILITY,True,2018
137,ELEC,LADWP - Harbor Generating Station,161 N Island Ave,Wilmington,CA,90744,33.770508,-118.265628,LG,1999-10-15,private,UTILITY,True,1999


Let's check the dataframe for duplicates.

In [3]:
# Check for duplicates
duplicates = ev_data[ev_data.duplicated()]
duplicates.head()

,Fuel Type Code,Station Name,Street Address,City,State,ZIP,Latitude,Longitude,Owner Type Code,Open Date,Access Code,Facility Type,EV Workplace Charging,Year
3974,ELEC,Jeff Schmitt Nissan,725 Alpha Rd,Beavercreek,OH,45434,39.708775,-84.026292,P,2012-01-31,private,CAR_DEALER,False,2012
4193,ELEC,Freedom Honda Sumter,2700 Broad St,Sumter,SC,29150,33.962204,-80.402858,P,2012-04-15,private,CAR_DEALER,False,2012
4408,ELEC,Don Franklin Nissan - Lexington,3360 Richmond Rd,Lexington,KY,40509,37.997336,-84.441827,P,2012-01-31,public,CAR_DEALER,False,2012
5536,ELEC,Fairfield Mitsubishi,421 Tunxis Hill Rd,Fairfield,CT,6825,41.176442,-73.230597,P,2011-12-01,private,CAR_DEALER,False,2011
6805,ELEC,City of San Dimas - Maintenance Yard,301 S Walnut Ave,San Dimas,CA,91773,34.104155,-117.803402,LG,2014-03-13,public,MUNI_GOV,False,2014


It looks like there are a few. How many?

In [4]:
# Count duplicates
duplicate_count = ev_data.duplicated().sum()
print(f'Total duplicate rows: {duplicate_count}')


Total duplicate rows: 30


I can't find any notes in the documentation about duplicates. Since there are a relatively small number of duplicates (30), and there is nothing to indicate that these duplicates are intentional, I'm going to drop these rows. 

In [5]:
# First, let's count the number of rows in the dataframe
num_rows = ev_data.shape[0]
print(f'Number of rows: {num_rows}')

Number of rows: 74682


In [6]:
# Drop duplicates
ev_data_cleaned = ev_data.drop_duplicates()

In [7]:
# Count again to verify that the rows were dropped
num_rows = ev_data_cleaned.shape[0]
print(f'Number of rows: {num_rows}')

Number of rows: 74652


### Step 3: Run Calculations

I want to get a sense for how many electric charging stations (or EV charging stations) have opened in each state, per year.

In [8]:
# Group by 'State' and 'Year' to get the count of chargers per state per year
chargers_per_state_per_year = ev_data_cleaned.groupby(['State', 'Year']).size().unstack(fill_value=0)

# Add any missing years with 0 chargers
all_years = pd.Index(range(ev_data_cleaned['Year'].min(), ev_data_cleaned['Year'].max() + 1))
chargers_per_state_per_year = chargers_per_state_per_year.reindex(columns=all_years, fill_value=0)

chargers_per_state_per_year

,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
State,,,,,,,,,,,,,,,,,,,,,
AK,0,0,0,0,0,0,0,0,0,0,...,2,0,5,7,4,15,15,15,4,3
AL,0,0,0,0,0,0,0,0,0,0,...,31,10,33,16,19,31,45,109,113,62
AR,0,0,0,0,0,0,0,0,0,0,...,6,5,18,13,19,12,50,80,122,26
AZ,0,0,0,0,0,0,0,0,0,0,...,45,45,33,49,76,80,344,264,246,140
CA,1,1,6,14,12,3,0,17,0,14,...,418,319,306,556,618,2862,6092,2712,2548,1254
CO,0,0,0,0,0,0,0,0,0,0,...,61,54,92,111,127,271,545,429,478,187
CT,0,0,0,0,0,0,0,0,0,0,...,44,71,22,32,54,39,60,163,435,267
DC,0,0,0,0,0,0,0,0,0,0,...,26,15,21,29,39,26,104,55,52,16
DE,0,0,0,0,0,0,0,0,0,0,...,7,2,16,5,6,22,57,41,53,35


An interesting find: it looks like 6,094 EV charging stations opened in California in 2021. That seems like a ton! Let's investigate.

In [9]:
# Filter ev_data to view charging stations opened in 2021 in California
ev_data_CA_2021 = ev_data_cleaned[(ev_data_cleaned['Year'] == 2021) & (ev_data_cleaned['State'] == 'CA')]

ev_data_CA_2021.head(20)

,Fuel Type Code,Station Name,Street Address,City,State,ZIP,Latitude,Longitude,Owner Type Code,Open Date,Access Code,Facility Type,EV Workplace Charging,Year
7434,ELEC,LA County Sheriff Communications Center,1277 N Eastern Ave,Los Angeles,CA,90063,34.052459,-118.175258,LG,2021-08-02,private,MUNI_GOV,True,2021
7441,ELEC,LA County Sheriff San Dimas Station,270 S Walnut Ave,San Dimas,CA,91773,34.105077,-117.802207,LG,2021-06-02,public,MUNI_GOV,False,2021
7444,ELEC,LA County Sheriff Walnut Station,21695 Valley Blvd,Walnut,CA,91789,34.028667,-117.833689,LG,2021-05-21,public,MUNI_GOV,False,2021
7644,ELEC,LA County DHS MLK Lot C,1720 E 120th St,Los Angeles,CA,90059,33.923781,-118.240951,LG,2021-08-20,private,HOSPITAL,True,2021
8538,ELEC,LA County DHS Harbor UCLA MFI Lot,1000 W Carson St,Torrance,CA,90502,33.828186,-118.290921,LG,2021-08-03,public,HOSPITAL,False,2021
14972,ELEC,Coronado Island Marriott Resort & Spa - Tesla ...,2000 2nd St,Coronado,CA,92118,32.694803,-117.166140,P,2021-12-15,public,HOTEL,False,2021
15194,ELEC,Haven Craft Kitchen + Bar - Tesla Destination,190 S. Glassell St,Orange,CA,92866,33.786281,-117.853549,P,2021-02-24,public,RESTAURANT,False,2021
23870,ELEC,TownePlace Suites - Clovis,580 W Shaw Ave,Clovis,CA,93612,36.807100,-119.721880,P,2021-01-27,public,HOTEL,False,2021
24276,ELEC,Hayward High School,1633 East Ave,Hayward,CA,94541,37.671830,-122.069135,NaN,2021-08-01,public,NaN,False,2021
30579,ELEC,HOWARD CDM THE EXCHANGE 3,17434 BELLFLOWER BLVD,BELLFLOWER,CA,90706.0,33.873680,-118.124105,NaN,2021-01-01,public,NaN,False,2021


I'm not immediately seeing any issues here. I'll check for duplicate dates.

In [10]:
# Count duplicates in the 'Open Date' column
duplicate_count_open_date = ev_data_CA_2021.duplicated(subset=['Open Date']).sum()
print(f'Total duplicate rows based on "Open Date": {duplicate_count_open_date}')


Total duplicate rows based on "Open Date": 5784


It looks like there are a lot of date duplicates. I'll check to see if any specific dates stand out as oft-repeated dates.

In [11]:
# Find the 10 most commonly repeated values in the 'Open Date' column
top_10_open_dates = ev_data_CA_2021['Open Date'].value_counts().head(10)
print("Top 10 most commonly repeated 'Open Date' values:")
print(top_10_open_dates)


Top 10 most commonly repeated 'Open Date' values:
2021-01-27    3989
2021-01-15     160
2021-10-28      84
2021-12-09      59
2021-05-11      52
2021-09-21      37
2021-07-13      30
2021-08-26      27
2021-05-30      26
2021-12-04      22
Name: Open Date, dtype: int64


Hm! It looks like almost 4,000 of the 2021 California EV charging stations have the same 'Open Date'. That doesn't seem right.  

Unfortunately, I haven't been able to find any information about this in the documentation. I double checked my analysis and API download, but I don't see any issues there either. I reached out to some folks connected to the data source, so I will update this if I hear back. 

Until then, I have a few options:
1. Assume the dates are accurate
2. Assume the year, at least, is accurate
3. Assume the dates are incorrect or inaccurate

For now, I'm going to go with option 2. My guess is that the exact dates are inaccurate, but hopefully, the year (2021) is correct.

Now, let's get back to calculating the rate of growth of electric charging stations per year.

In [12]:
# Calculate the cumulative sum to get the total number of chargers up to each year
cumulative_chargers = chargers_per_state_per_year.cumsum(axis=1)

cumulative_chargers.head(10)

,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
State,,,,,,,,,,,,,,,,,,,,,
AK,0,0,0,0,0,0,0,0,0,0,...,3,3,8,15,19,34,49,64,68,71
AL,0,0,0,0,0,0,0,0,0,0,...,79,89,122,138,157,188,233,342,455,517
AR,0,0,0,0,0,0,0,0,0,0,...,35,40,58,71,90,102,152,232,354,380
AZ,0,0,0,0,0,0,0,0,0,0,...,115,160,193,242,318,398,742,1006,1252,1392
CA,1,2,8,22,34,37,37,54,54,68,...,1220,1539,1845,2401,3019,5881,11973,14685,17233,18487
CO,0,0,0,0,0,0,0,0,0,0,...,172,226,318,429,556,827,1372,1801,2279,2466
CT,0,0,0,0,0,0,0,0,0,0,...,175,246,268,300,354,393,453,616,1051,1318
DC,0,0,0,0,0,0,0,0,0,0,...,55,70,91,120,159,185,289,344,396,412
DE,0,0,0,0,0,0,0,0,0,0,...,20,22,38,43,49,71,128,169,222,257


In [13]:
# Calculate the percentage change over the last 5 years
    # Use axis=1 to calculate along the columns
growth_rate = cumulative_chargers.pct_change(periods=5, axis=1).iloc[:, -1] # Use iloc[:, -1] to select the last column

growth_rate

State
AK     2.736842
AL     2.292994
AR     3.222222
AZ     3.377358
CA     5.123551
CO     3.435252
CT     2.723164
DC     1.591195
DE     4.244898
FL     2.482759
GA     2.892794
HI     1.361702
IA     2.990741
ID     1.589474
IL     2.331868
IN     2.561497
KS     2.210526
KY     2.410000
LA     2.000000
MA     7.239130
MD     2.061807
ME     1.893855
MI     4.187302
MN     2.547619
MO     3.189759
MS     1.652778
MT     1.916667
NC     2.562738
ND     3.521739
NE     3.421875
NH     1.605263
NJ     3.891304
NM     4.375000
NV     2.772222
NY     3.597889
OH     3.359036
OK     2.579439
OR     2.117021
PA     3.318376
PR    16.000000
RI     2.926316
SC     1.905579
SD     1.820513
TN     2.685714
TX     3.370286
UT     3.260504
VA     2.159649
VT     1.528090
WA     2.511471
WI     2.497512
WV     1.361111
WY     1.222222
Name: 2024, dtype: float64

Upon reviewing the data, it looks like Puerto Rico has an extremely high growth rate, due to the very small number of charging stations there in general. I'm going to exclude Puerto Rico from the data going forward, as its growth rate (based on very few charging stations) will be an outlier compared to other states.

In [14]:
# Exclude 'PR' from the growth_rate
growth_rate_filtered = growth_rate[growth_rate.index != 'PR']

# Reset the index
growth_rate_filtered = growth_rate_filtered.reset_index()

# Correct the column title
growth_rate_filtered.rename(columns={2024: 'Increase in Chargers from 2019 to 2024'}, inplace=True)

print(growth_rate_filtered)

   State  Increase in Chargers from 2019 to 2024
0     AK                                2.736842
1     AL                                2.292994
2     AR                                3.222222
3     AZ                                3.377358
4     CA                                5.123551
5     CO                                3.435252
6     CT                                2.723164
7     DC                                1.591195
8     DE                                4.244898
9     FL                                2.482759
10    GA                                2.892794
11    HI                                1.361702
12    IA                                2.990741
13    ID                                1.589474
14    IL                                2.331868
15    IN                                2.561497
16    KS                                2.210526
17    KY                                2.410000
18    LA                                2.000000
19    MA            

Now I have a growth rate value for every state! Interestingly, it looks like Massachusetts has the highest rate, while Wyoming has the lowest rate.

### Step 4: Exports for Visualization

I want to create some visualizations to help me understand the results of my analysis, so I'm going to export some of my dataframes as CSV files to use in Tableau. 

In [15]:
# Export the cumulative_chargers DataFrame to a CSV file
cumulative_chargers.to_csv('EV_Charging_Stations_Cumulative.csv', index=True)

In [16]:
# Export the ev_data_cleaned DataFrame to a CSV file
ev_data_cleaned.to_csv('EV_Charging_Stations_Cleaned.csv', index=False)


In [17]:
# Export the growth_rate_filtered DataFrame to a CSV file
growth_rate_filtered.to_csv('EV_Charging_Stations_Growth_Rate_per_State.csv', index=True)
